In [1]:
""" https://github.com/sbarratt/inception-score-pytorch/blob/master/inception_score.py """
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data

from torchvision.models.inception import inception_v3
from torchvision import transforms

import numpy as np
from scipy.stats import entropy

normalize = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# Load inception model
inception_model = inception_v3(pretrained=True, transform_input=False).to(device)

In [8]:
def inception_score(imgs, batch_size=32, resize=False, splits=1):
    """Computes the inception score of the generated images imgs
    imgs -- Torch dataset of (3xHxW) numpy images normalized in the range [-1, 1]
    batch_size -- batch size for feeding into Inception v3
    splits -- number of splits
    """
    N = len(imgs)

    assert batch_size > 0
    assert N > batch_size

    # Set up dataloader
    dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size)
    
    inception_model.eval();
    up = nn.Upsample(size=(299, 299), mode='bilinear').float().to(device)
    def get_pred(x):
        if resize:
            x = up(x)
        x = inception_model(x)
        return F.softmax(x).data.cpu().numpy()

    # Get predictions
    preds = np.zeros((N, 1000))

    for i, batch in enumerate(dataloader, 0):
        batch = batch.float().to(device)
        batchv = Variable(batch)
        batch_size_i = batch.size()[0]

        preds[i*batch_size:i*batch_size + batch_size_i] = get_pred(batchv)

    # Now compute the mean kl-div
    split_scores = []

    for k in range(splits):
        part = preds[k * (N // splits): (k+1) * (N // splits), :]
        py = np.mean(part, axis=0)
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]
            scores.append(entropy(pyx, py))
        split_scores.append(np.exp(np.mean(scores)))

    return np.mean(split_scores), np.std(split_scores)

In [9]:
# %matplotlib inline
# import matplotlib.pyplot as plt
import os
import glob
import cv2
def load_images_from_path(images_path, width=256):
    # loads all images into memory (this might require a lot of RAM!)
    image_list = glob.glob(os.path.join(images_path, '*.jpg'))

    images = np.zeros((len(image_list), 3, width, width))
    i = 0
    for img_fn in image_list:
        img = cv2.imread(img_fn, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (width, width), interpolation = cv2.INTER_AREA)
        img = img[:,:,::-1] # BGR to RGB
        img = img / 255. # 0-1
        img = normalize(img) # Normalize
        images[i] = img
        i += 1
    return images

In [10]:
PARENT_DIR = 'fid'
labels = ['GAN', 'L2', 'L1*', 'CM', 'CM + L1*']
dir_labels = ['gan', 'l2', 'l1', 'cm', 'cml1']
inds_to_save = ['final'] + [20, 50, 100, 300]


for painting_ind in inds_to_save:
    for dir_label in dir_labels:
        gen_image_dir = os.path.join(PARENT_DIR, str(painting_ind), dir_label)
        
        imgs = load_images_from_path(gen_image_dir, width=299)
        
#         plt.imshow(np.transpose(imgs[0], (1,2,0)))
#         plt.show()
                
        means, stds = inception_score(imgs, batch_size=8, resize=False, splits=5)
        
        print(painting_ind, dir_label, means, stds)
        

C:\Program Files\Python36\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


final gan 3.0452471956212603 0.08291245330747174
final l2 3.0490264758739984 0.14296767185980672
final l1 3.452455208449917 0.0924104153592888
final cm 3.008645696573719 0.0385964334521512
final cml1 3.4733041640342357 0.04700355283420716
20 gan 2.067051576149812 0.04347783370869842
20 l2 2.2007180896651612 0.022177928665282447
20 l1 2.555161535262069 0.04286607222043453
20 cm 2.0002639159581816 0.061534563172821424
20 cml1 2.161550420584192 0.06397792961397562
50 gan 1.9836590849696918 0.021747310565945695
50 l2 2.460179597369037 0.07752073113340555
50 l1 2.541862396321299 0.13191516826046504
50 cm 2.1315592320439363 0.06206673327318728
50 cml1 2.394289023377452 0.07174853153242801
100 gan 2.2615243486862466 0.05144036416501071
100 l2 2.7862814005843193 0.07674621308113994
100 l1 2.9307318068103902 0.058639127017219454
100 cm 2.6182283436320706 0.07340186508533178
100 cml1 2.791565134636432 0.026414792475662658
300 gan 2.4848729255434407 0.04484469781809107
300 l2 3.1322368882965494 0